# (day4) Section 3: 軽量化 - 高速化技術

本書は、「深層学習後編（day4）レポート」の、「Section 3: 軽量化 - 高速化技術」についてのものです。


## 1. 要点まとめ


本章では、「分散深層学習」で学習自体の高速化を、<br/>
「モデルの軽量化」でモデル自体を高速に学習可能とするための方法を扱う。


<!-- - page. 37 - 74

- 分散深層学習
  - データ並列化
    - 同期型
    - 非同期型
  - モデル並列化
  - GPU による高速化
    - 処理装置の種類
    - GPGPU 開発環境
- モデルの軽量化
  - 量子化 ( Quantization )<br/>
    表にする。
    - 利点と欠点
      - 利点
        - 計算の高速化
        - 省メモリ化
      - 欠点
        - 精度の低下<br/>
          含む "極端な量子化"
  - 蒸留 ( Distillation )<br/>
    概要、モデルの簡約化
    - 教師モデルと生徒モデル
    - 蒸留の利点 ( 効果 )
  - Pruning ( 剪定 ) <br/>
    - 計算の高速化
    - ニューロンの削減<br/>
      含む "ニューロン数と精度" -->


### 分散深層学習

計算資源が増大する傾向にある深層学習を行うため、複数のワーカーを使用し、並列で学習を行う。

並列化の技術は、以下の通り。

- データ並列化
- モデル並列化
- GPU による高速化


#### データ並列化

データを分割して、各ワーカーに並列で学習させる。
元のモデルを親モデル、コピーして各ワーカーで学習させるモデルを子モデルと言う。

各ワーカーが子モデルで学習した結果を、
親モデルに更新する方法によって、以下の 2 種類に分類される。

- 同期型<br/>
  全ワーカーの学習を待ち、全てのパラメータの勾配が産出されたら、
  親モデルのパラメータに更新する。
- 非同期型<br/>
  各ワーカーが非同期に、独立して学習を進める。<br/>
  学習された子モデルを、パラメータのやり取りをするパラメータサーバに、
  Push して、<br/>
  学習を始めるときは、改めてパラメータサーバから最新の子モデルを
  Pop して学習を進める。

更新を管理しやすく、精度が良い傾向にある同期型を使うことが多いが、<br/>
ワーカーを配置するデバイスの種類など、ケースによって使い分ける。


#### モデル並列化

親モデルを分割して、各ワーカーで並列で学習する。

分割のしかたは、以下の 2 通り。

- 層の区切りで分割
- ネットワークの分岐から分割<br/>
  近年はこの場合が多い。

モデルのパラメータ数が多いほど、効率が良い。
ネットワークなどの、ワーカー間のオーバーヘッドを相殺するだけのモデルの大きさがあるとメリットが大きくなる。


#### GPU による高速化

演算処理装置に、並列処理が得意で行列計算に向いている GPU を利用して高速化する。

GPU の元々の使用目的は、グラフィック演算。グラフィック以外の用途で使用される GPU のことを、 GPGPU ( General-purpose on GPU ) と言う。


GPGPU を使用するための開発環境には、 CUDA と OpenCL がある。

現時点では、 OpenCL はほとんど使用されず、 NVIDIA 社製 GPU において Deep Learning 用に提供されている CUDA を使用することが多い。


### モデルの軽量化

モデルの精度を維持して、パラメータや演算回数を低減する手法。
モバイル、 IoT 機器で有効。

モデルの軽量化手法は、以下の通り。

- 量子化 ( Quantization )
- 蒸留 ( Distillation )
- Pruning ( 剪定 )


#### 量子化 ( Quantization )

パラメータの精度を、 64 bit 浮動小数点から 32 bit 、または 16 bit の下位の精度に下げること。

大量のパラメータにおいて、パラメータの精度を下げるとその分のデータ量差削減となり、
メモリ使用量、演算量を低減できる。


量子化の利点と欠点は、以下の通り。

- 利点
  - 計算の高速化<br/>
    倍精度演算 ( 64bit ) から、単精度演算 ( 32 bit ) にすることで、
    約 2 倍に高速化できる。
  - 省メモリ化<br/>
    下げた bit 数分、メモリを節約できる。 ( 16 / 64 = 1 / 4 )

- 欠点
  - 精度の低下<br/>
    精度を下げた分、浮動小数点の有効桁数が減るので、少数の有効桁が小さくなる。
    結果、モデル ( 重み ) の表現力が低下する。

実際問題としては、倍精度を単精度にしてもほぼ精度は変わらない。


#### 蒸留 ( Distillation )

精度の高い大きなモデルの知識 ( 学習結果の重み ) を使って、軽量な ( 小さな ) モデルを作成する。


教師モデルと生徒モデルを使う。

- 教師モデル<br/>
  精度の高い大きなモデル
- 生徒モデル<br/>
  教師モデルの知識を使って学習させる、小さなモデル


##### 教師モデルと生徒モデル ( 生徒モデルの学習方法 )

- 教師モデルの重みは更新しない。 ( 学習が終わったお手本 )
- 教師モデルと生徒モデルの両方に訓練データを入力する。
- 両方の出力と訓練データの正解の誤差を得て、<br/>
  生徒モデルの重みへ更新する。


- NOTE: Hints Training
  - https://arxiv.org/pdf/1412.6550.pdf<br/>
    "Figure 1: Training a student network using hints" より。

教師モデルに、教師のヒントとしての $ W_{Hint} $ の重みの中間層を、
生徒モデルに、指導を受ける $ W_{Guided} $ の重みの中間層を持つ。

全体 ( 蒸留 ) の誤差とは別に、<br/>
教師ヒントの中間層と、指導を受ける生徒の中間層の出力の誤差を、
指導を受ける $ W_{Guided} $ の重みに更新しながら、学習を進める。


#### Pruning ( 剪定 )

モデルの精度に寄与が少ないニューロンを削減する。
( 不要な枝: ニューロンを剪定する )

削除したニューロンの分、モデルを軽量化できて、結果、高速化が見込まれる。

page. 73 左表、 1 行目の様に約半分のパラメータにしても、91.66 % の精度か出ている。

- NOTE: 元の精度
  - https://db-event.jpn.org/deim2017/papers/62.pdf<br/>
  page. 7 <br/>
  > CaffeNet のパラメータ削減前の Oxford
102 category flower dataset のテストデータにおける認識精度
（top-1）は 91.82%

パラメータを半分にしても、 0.16 % しか精度は落ちていない。

上記引用元より、パラメータの削減には、以下の 2 種類がある。

> 結合単位でのパラメータの削減

ユニット ( ニューロン ) 間の結合を減らす。

> ユニット単位でのパラメータの削減

ユニット ( ニューロン ) を減らす。

ネットワークによって、どちらが有効か異なるとのこと。
先の CaffeNet は、「ユニット単位でのパラメータの削減」によるもの。
